In [1]:
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE

In [2]:
with open("songData.json", "r") as f:
	song_transcripts = json.load(f)

songs = list(song_transcripts.values())
print(songs[0].keys())

dict_keys(['page_views', 'lyrics', 'url', 'artist', 'release_date', 'title'])


In [3]:
vectorizer = TfidfVectorizer(stop_words = "english", max_df = 0.7, min_df = 75)
lyric_matrix = vectorizer.fit_transform([song["lyrics"] for song in songs]).transpose()

u, s, v = svds(lyric_matrix, k=100)
words_compressed, _, docs_compressed = svds(lyric_matrix, k = 40)
docs_compressed = docs_compressed.transpose()

print(words_compressed.shape)
print(docs_compressed.shape)

word_to_index = vectorizer.vocabulary_
print(word_to_index)
index_to_word = {i:t for t,i, in word_to_index.items()}
words_compressed = normalize(words_compressed, axis =1)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1039: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


(2118, 40)
(4544, 40)
{'nles': 1192, 'sample': 1616, 'mes': 987, 'nshow': 1279, 'need': 1114, 'pink': 1431, 'drugs': 450, 'ngirl': 1139, 'games': 607, 'cake': 220, 'windows': 2068, 'butter': 215, 'little': 904, 'spread': 1790, 'nlivin': 1200, 'point': 1458, 'till': 1918, 'lick': 887, 'se': 1637, 'vibe': 1999, 'bag': 70, 'flame': 555, 'rain': 1527, 'tre': 1949, 'callin': 223, 'nshit': 1276, 'use': 1993, 'nwent': 1346, 'ear': 461, 'higher': 723, 'bury': 206, 'automatic': 61, 'swing': 1865, 'trunk': 1965, 'label': 848, 'enemy': 477, 'ay': 65, 'went': 2049, 'nsee': 1274, 'fifth': 538, 'hall': 676, 'flesh': 559, 'dig': 405, 'ballin': 74, 'shed': 1673, 'rubber': 1603, 'major': 956, 'knowin': 841, 'crooked': 353, 'math': 973, 'thank': 1894, 'meat': 981, 'read': 1542, 'listening': 902, 'mad': 951, 'flip': 562, 'faire': 501, 'best': 105, 'couple': 337, 'man': 964, 'nred': 1265, 'nmaking': 1211, 'wish': 2074, 'making': 960, 'clean': 290, 'fair': 500, 'ntill': 1314, 'makin': 959, 'lap': 857, 'res

In [4]:
tsne = TSNE(verbose=1)
projected_docs = tsne.fit_transform(docs_compressed)

docs_compressed = normalize(docs_compressed, axis=1)

def closest_songs(project_index_in, k=10):
	sims = docs_compressed.dot(docs_compressed[project_index_in,:])
	asort = np.argsort(-sims)[:k+1]
	return [(songs[i]["title"], sims[i]/sims[asort[0]]) for i in asort[1:]]

for i in range(10):
	print(songs[i]["title"])
	for title, score in closest_songs(i):
		print("{}:{:.3f}".format(title[:40], score))
	print()

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 4544
[t-SNE] Computed conditional probabilities for sample 2000 / 4544
[t-SNE] Computed conditional probabilities for sample 3000 / 4544
[t-SNE] Computed conditional probabilities for sample 4000 / 4544
[t-SNE] Computed conditional probabilities for sample 4544 / 4544
[t-SNE] Mean sigma: 0.022457
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.381635
[t-SNE] Error after 125 iterations: 1.381635
4, 3, 2, 1
313:0.666
Tres Leches (Triboro Trilogy):0.618
Pass the Courvoisier Part II:0.617
Mighty O:0.593
Full Clip:0.584
Jump Around:0.569
Freestyle Or Written:0.566
What's Your Fantasy (Remix):0.557
Shame On A Nigga:0.555
Instigator:0.554

My Favorite Ladies
Passin' Me By:0.831
Seatbelt Hands:0.817
Blue Dolphin:0.813
Billie Jean:0.792
90210:0.791
Dr. Aden:0.784
Prom Queen:0.775
Miami 2 Ibiza:0.774
Dirty Diana:0.768
Exxxes:0.768

S

In [8]:
print(docs_compressed[1,:])

[-0.11891473 -0.26548626  0.21844508 -0.01728063 -0.06813676 -0.14546947
 -0.20154515 -0.07669739 -0.1464848   0.00771363 -0.00548259  0.0862444
 -0.17161614 -0.25266379 -0.02505389 -0.16431443 -0.11432773 -0.03021782
 -0.24826639  0.20361476  0.06523567 -0.18614098  0.03771754  0.05361473
 -0.1127187   0.41676542 -0.15419024  0.16910841  0.02351737  0.25466298
 -0.03605542 -0.00538426  0.011335   -0.04147829 -0.28644682 -0.05429956
  0.06326918 -0.21121065  0.02032675  0.2085351 ]


In [9]:
query_mat = vectorizer.fit_transform(["Lets hang out tomorrow, when are you free?"]).transpose()


ValueError: max_df corresponds to < documents than min_df